In [1]:
!pip install -q keras

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

matplotlib 1.3.1 requires nose, which is not installed.
matplotlib 1.3.1 requires tornado, which is not installed.
Cannot uninstall 'numpy'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
torch-0.4.1-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10)

Using TensorFlow backend.


In [3]:
import numpy as np

def vectorize_sequences(sequences, dimension=5000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence]=1
  return results


In [4]:

imdb_training = vectorize_sequences(train_data)
imdb_testing = vectorize_sequences(test_data)

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chang_nina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from collections import Counter
def parse_amazon_data(lines):
    '''
    :param lines: a list of string... each line being a review + label
    :return: a list of [review (string), label (int)]
    '''
    
    # iterate through lines, create list of labels & list of reviews
    
    lines = [line.lower() for line in lines if len(line) >10]
    reviews = [0 for i in range(len(lines))]
    labels = [0 for i in range(len(lines))]
    all_words = []
    
    data = [0 for i in range(len(lines))]
               
    
    for i in range(len(lines)):
      label, review_str = lines[i].split(" ", 1)
      review_str = ' '.join([word for word in review_str.split() if word not in stop])
      all_words += [word for word in review_str.split() if word not in stop]
      labels[i] = int(label[9])-1
      reviews[i] = review_str
    
    print("done")
    
#     count = Counter(word for line in reviews for word in line.split())
    count = Counter(all_words)
    print("meow")
    
    # extract most common 10000 words
    
    common = [word for (word, count) in count.most_common(10)]
    common_set = set(common)
    print("mooooo")
    
    i = 0
    for r in reviews:
      r_words = set(r.split())
      # find union between r_words and common --> find index
#       data[i] = [i for i in range(len(common)) if common[i] in r_words]
      intersect = r_words.intersection(common_set)
      data[i] = [common.index(w) for w in intersect]
      i += 1
      if i % 100000 == 0:
        print(i)
    
    print("whee")
     
    return data, labels

In [7]:
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

# amazon_training = "./data/train.ft.txt.bz2"
# amazon_testing = "./testfrom google.colab import drive
# drive.mount('/content/gdrive').ft.txt.bz2"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
amazon_training = "./data/train.ft.txt.bz2"
amazon_testing = "./data/test.ft.txt.bz2"

In [8]:
amazon_training_lines = ""
with open("./data/train.ft.txt", 'r') as f:
  amazon_training_lines = f.readlines()

In [9]:
amazon_testing_lines = []

with open("./data/test.ft.txt", 'r') as f:
  amazon_testing_lines = f.readlines()

In [10]:
amazon_test_data, amazon_test_labels = parse_amazon_data(amazon_testing_lines)

done
meow
mooooo
100000
200000
300000
400000
whee


In [ ]:
amazon_training_data, amazon_training_labels = parse_amazon_data(amazon_training_lines)


done
meow
mooooo


In [ ]:
amazon_testing = vectorize_sequences(amazon_test_data)


In [ ]:
amazon_training = vectorize_sequences(amazon_training_data)

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(5000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
imdb_history = model.fit(imdb_training, train_labels, epochs=20, validation_data=(imdb_testing, test_labels), batch_size=512, shuffle=True)

In [ ]:
imdb_history1 = model.fit(imdb_training, train_labels, epochs=20, validation_data=(amazon_testing, amazon_test_labels), batch_size=512, shuffle=True)

In [ ]:
plt.plot(x1, imdb_history.history['acc']) 
plt.plot(x1, imdb_history1.history['val_acc']) 
plt.plot(x1, imdb_history.history['val_acc']) 


plt.legend(['IMDB training accuracy', 'Amazon testing accuracy', 'IMDB testing accuracy'], loc='lower right')
plt.title('Accuracy of Naive Bayes Model trained on IMDB')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.show()

In [ ]:
amazon_history = model.fit(amazon_training, amazon_training_labels, epochs=20, validation_data=(imdb_testing, test_labels), batch_size=512, shuffle=True)
amazon_history1 = model.fit(amazon_training, train_labels, epochs=20, validation_data=(amazon_testing, amazon_test_labels), batch_size=512, shuffle=True)

In [ ]:
plt.plot(x1, amazon_history.history['acc']) 
plt.plot(x1, amazon_history1.history['val_acc']) 
plt.plot(x1, amazon_history.history['val_acc']) 


plt.legend(['Amazon training accuracy', 'Amazon testing accuracy', 'IMDB testing accuracy'], loc='lower right')
plt.title('Accuracy of NN trained on IMDB')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(imdb_training_acc)


METHOD #2 but not sure how we can use this if input length of training != input length of testing

In [0]:
amazon_training = "./train.ft.txt.bz2"
amazon_testing = "./test.ft.txt.bz2"
imdb_training = "./aclImdb/train"
imdb_testing = "./aclImdb/test"


In [0]:
amazon_training_file = bz2.BZ2File(amazon_training,'r')
amazon_training_lines = amazon_training_file.readlines()

In [0]:
def parse_amazon_data(lines):
    '''
    :param lines: a list of string... each line being a review + label
    :return: a list of [review (string), label (int)]
    '''
    review_and_ratings_list=[]
    for i in range(len(lines)):
        line_text = str(lines[i])
        label, review_str = line_text.split(" ", 1)
        label = int(label[-1]) - 1         # label with 1 and 0
        review_and_ratings_list.append([review_str, label])
    return review_and_ratings_list

In [0]:
def parse_imdb_data(file_path):
    '''
    :param lines: a list of string... each line being a review + label
    :return: a list of [review (string), label (int)]
    '''
    review_and_ratings_list=[]
    
    for folder_name in os.listdir(file_path):
      if folder_name == "pos" or folder_name == "neg":
        if folder_name == "pos":
          label = 1
        elif folder_name == "neg":
          label = 0
          
          
        for file_name in os.listdir(file_path + "/" + folder_name):
          with open(file_path + "/" + folder_name + "/" + file_name, 'r', errors='ignore') as f:
            review_and_ratings_list.append([f.read(), label])
    return review_and_ratings_list

In [0]:
def remove_punc(s):
    table = str.maketrans({key: None for key in string.punctuation})
    return s.translate(table)

In [0]:
def generate_x_and_y(review_and_ratings_list):
    data = pd.DataFrame(review_and_ratings_list, columns=['review','rating'])
    data['word_count'] = data['review'].str.lower().str.split().apply(len)
    data['review'] = data['review'].apply(remove_punc)
    data = data[:][data['word_count']<=25]
    
    stop_word = text.ENGLISH_STOP_WORDS
    count_vector = CountVectorizer(stop_words = stop_word, min_df=.0001, lowercase=1)
    x = csr_matrix(count_vector.fit_transform(data['review'].values)).todense()
    #y = csr_matrix(data['rating'].values).todense()
    y = data['rating'].values
    return (x, y)

In [0]:
#amazon_training_review_and_ratings_list = parse_amazon_data(amazon_training_lines)

imdb_testing_review_and_ratings_list = parse_imdb_data(imdb_testing)

In [0]:
#train_x, train_y = generate_x_and_y(amazon_training_review_and_ratings_list)
test_x, test_y = generate_x_and_y(imdb_testing_review_and_ratings_list)

In [0]:
import numpy as np

padded = np.pad(test_x, (0,9390)

(37, 288)

In [0]:
print(type(test_x))

<class 'numpy.matrixlib.defmatrix.matrix'>


In [0]:
model1=  Sequential()
model1.add(Dense(1000,input_shape=(9678,),activation='relu'))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
hist = model1.fit(train_x, train_y, epochs=6, batch_size=128, verbose=1)

In [0]:
model1.evaluate(test_x, test_y, batch_size=128)

In [0]:
def parse_amazon_data(lines):
    '''
    :param lines: a list of string... each line being a review + label
    :return: a list of [review (string), label (int)]
    '''
    review_and_ratings_list=[]
    for i in range(len(lines)):
        line_text = str(lines[i])
        label, review_str = line_text.split(" ", 1)
        label = int(label[-1]) - 1         # label with 1 and 0
        review_and_ratings_list.append([review_str, label])
    return review_and_ratings_list

In [0]:
amazon_testing_review_and_ratings_list = parse_amazon_data(amazon_testing_lines)


<zipfile.ZipFile [closed]>


In [0]:
model1.evaluate(X_test, y_test, batch_size=128)


NameError: ignored